<b><font size=5>貪欲法＆動的計画法による<br>
複数ナップザック問題の近似解推定</font></b>

In [1]:
import time
import random
import numpy as np
import pandas as pd
from ortoolpy import knapsack

### ○データの準備

In [2]:
# サンプルデータの読み込み
df_sample = pd.read_csv('sample_10*5000.csv', sep='\t')
df_sample.head(3)

,id,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,...,rating_0,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9
0,1,380,385,389,375,387,385,382,378,385,...,4.1,3.3,4.0,3.5,3.8,4.5,4.1,3.8,3.8,3.8
1,2,943,947,961,960,960,947,950,940,944,...,10.1,8.9,9.3,10.3,9.4,9.5,9.9,9.4,8.4,9.1
2,3,980,980,987,982,974,982,981,982,975,...,10.4,8.7,10.3,9.8,9.3,9.0,9.8,9.8,9.8,9.8


In [3]:
# clientの数。 
# １行（id）分引いて2で割れば、クライアントの数になる。
client_num = int((len(df_sample.columns)-1)/2)
print('クライアントの数:', client_num)

クライアントの数: 10


In [4]:
# 初めにクライアントが持っていた枠をランダムに決める（別のデータに対応できるかを見るため。）
# そのため、df_inputをモデルにいれる。
df_input = df_sample.sample(frac=1).reset_index(drop=True)
df_input.head(3) # ランダムになっていることの確認。

,id,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,...,rating_0,rating_1,rating_2,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9
0,672,1122,1122,1113,1113,1105,1098,1090,1096,1102,...,11.6,10.7,10.7,11.1,10.2,10.7,11.7,11.1,11.9,12.7
1,4362,298,307,313,313,319,327,322,328,329,...,2.5,2.1,1.7,2.3,2.3,3.3,3.3,4.3,3.9,4.1
2,107,900,908,913,913,913,913,915,915,905,...,8.9,8.5,8.9,9.0,8.8,9.5,9.5,9.4,9.2,8.3


In [5]:
# クライアト0, 1, 2の元々持っていた枠をランダムに決める。
length = len(df_input) # データフレームの長さ
bps = [0 for i in range(client_num+1)] # Break Points
bp = 0
original_prices = [0 for i in range(client_num)] # 元々持っていた枠の価格和。
original_points = [0 for i in range(client_num)] # 元々持っていた枠の視聴率和。
original_idses = [[] for i in range(client_num)] # 元々持っていた枠のid。

for i in range(client_num):
    bp += length//client_num
    bps[i+1] = bp
    df_original = df_input[bps[i]:bps[i+1]] # 各クライアントが元々持っていた枠のデータフレーム 
    original_prices[i] = sum(df_original['price_' + str(i)]) # 元々持っていた価格の和。
    original_points[i] = sum(df_original['rating_' + str(i)]) # 元々持っていた視聴率の和。
    original_idses[i] = df_original['id'].tolist()

In [6]:
print(original_prices) # 元々持っていた枠の価格和。
print(original_points) # 元々持っていた枠の視聴率和。

[434091, 412386, 421122, 420708, 421766, 421947, 411318, 428801, 425818, 434951]
[4354.9, 4131.699999999999, 4201.799999999999, 4201.500000000003, 4209.500000000001, 4220.600000000004, 4105.2, 4291.0, 4260.400000000001, 4382.0]


In [7]:
# value=「視聴率/号数（価格）」を求める。(これを比較して、貪欲に取り出す。)
for i in range(client_num):
    df_input['value_'+str(i)] = df_input['rating_'+str(i)]/df_input['price_'+str(i)]
    
df_input.head(3) # valueのできていることの確認。

,id,price_0,price_1,price_2,price_3,price_4,price_5,price_6,price_7,price_8,...,value_0,value_1,value_2,value_3,value_4,value_5,value_6,value_7,value_8,value_9
0,672,1122,1122,1113,1113,1105,1098,1090,1096,1102,...,0.010339,0.009537,0.009614,0.009973,0.009231,0.009745,0.010734,0.010128,0.010799,0.011431
1,4362,298,307,313,313,319,327,322,328,329,...,0.008389,0.006840,0.005431,0.007348,0.007210,0.010092,0.010248,0.013110,0.011854,0.012239
2,107,900,908,913,913,913,913,915,915,905,...,0.009889,0.009361,0.009748,0.009858,0.009639,0.010405,0.010383,0.010273,0.010166,0.009263


***
***

### ○ここから関数定義（アルゴリズムの中身）

In [8]:
# 超貪欲法
def Greedy_Algorithm(df, require_points, got_idses, num):
    require_ratings = list(require_points) # 参照してしまい、値が変わってしまうのを防ぐため。
    achieved_num=0 # 目的を達成した（元々の視聴率和を超えた）クライアントの数。    
    value_col_names = ['value_'+str(i) for i in range(num)] # valueのカラム名（ここから目的を達成したクライアントを取り除いていく。）
    
    while achieved_num < num:
        df['Max_value'] = df.loc[:, value_col_names].max(axis=1) # (残っている)クライアントの価値の中で最も高い値。
        df['Max_value_client'] = df.loc[:, value_col_names+['Max_value']].apply(lambda x : list(x[x==x['Max_value']].index)[:-1],axis=1)# 最も高い値をつけているクライアント（複数）
        df['Max_value_client'] = df['Max_value_client'].apply(lambda x: random.choice(x).replace('value_', '')) # その中からクライアントを一つ選ぶ。
        df = df.sort_values(by='Max_value', ascending=False).reset_index(drop=True) # その価値でソートする。
        
        for i in range(len(df)):
            name = int(df.at[0, 'Max_value_client']) # 最も高い値をつけているクライアントの名前。（というか番号。） 
            require_ratings[name] -= df.at[0, 'rating_'+str(name)] # 必要な視聴率和からその枠の視聴率を引く。
            got_idses[name] += [df.iat[0, 0]] # 獲得した枠のidを記録する。([df.at[0, 'id']])
            df = df[1:].reset_index(drop=True) # dfを下っていく。
            if min(require_ratings) < 0: # どこかのクライアントが、目的を達成したら、一回終わり。
                break

        achieved_client = require_ratings.index(min(require_ratings)) # 目的を達成したクライアント名。（番号）
        require_ratings[achieved_client] = 0 # 目的を達成したら、0にする。（今後のため。）
        value_col_names.remove('value_'+str(achieved_client)) # 目的を達成したクライアントは、除く。
        achieved_num += 1 # チェックポイント＝目的を達成したクライアントの数。
            
    return got_idses # ぞれぞれのクライアントが獲得したidを返す。

In [9]:
# 動的計画法
def Dynamic_Programming(df, initial_prices, got_prices, rate, num):
    got_idses = [[] for i in range(num)] # 最終的に手にいれることのできたidのリスト
    capacities = list(np.array(initial_prices) * rate - np.array(got_prices))
    while len(df)>0:
        knapsack_indexes = [[] for i in range(num)] # ナップサックに入れたindexのリスト。
        add_indexes = [[] for i in range(num)] # 結果的に手にいれることのできたindexのリスト。
        
        for i in range(num):
            size = list(np.array(df['price_' + str(i)]))
            weight = list(np.array(df['rating_' + str(i)]))
            capacity = capacities[i]
            knapsack_indexes[i] = knapsack(size, weight, capacity)[1]
            
        chosen_index_dict = dict() # 一回のナップサックで選ばれたindexを格納する。
        for index in range(len(df)): 
            values = [] # そのindexを選択したクライアントの名前（番号）を格納する。
            for i in range(num):
                if index in knapsack_indexes[i]:
                    values.append(i) # 選択していたら加える。
            if values: # 選択したクライアントがいれば
                value = random.choice(values) # その中から一人選んで
                chosen_index_dict[index] = value # indexとともに格納する。
                
        if len(chosen_index_dict)==0: # もしどのクライアントもidを選ばなかったら
            return got_idses
                
        for i in range(client_num):
            add_indexes[i] = [index for index, client in chosen_index_dict.items() if client == i] # クライアントごとに、獲得した枠のindex
            got_idses[i] += df.query('index in ' + str(add_indexes[i]))['id'].values.tolist()
            capacities[i] -= sum(df.query('index in ' + str(add_indexes[i]))['price_' + str(i)]) # キャパシティから引いていく。

        extraction_ids = Flatten_dual(add_indexes) # 今回獲得されたidの集合
        df = df.query('index not in ' + str(extraction_ids)).reset_index(drop=True)
    
    return got_idses

In [10]:
# 手に入れた枠のidのリストを渡せば、手に入れた枠の価格と視聴率和を返す関数。
# Check Prices and Points
def Check_pp(df, got_idses, num):
    got_prices = [0 for i in range(num)]
    got_points = [0 for i in range(num)]
    for i in range(num):
        df_got = df[df['id'].apply(lambda x:x in got_idses[i])]
        got_prices[i] = sum(df_got['price_' + str(i)])
        got_points[i] = sum(df_got['rating_' + str(i)])
    return got_prices, got_points    


# 手に入れた枠のidのリストを渡せば、残ったデータフレームを返す関数。
def Check_remainingdf(df, got_idses):
    all_got_ids = Flatten_dual(got_idses) # クライアントが獲得したアカウント全てを足し合わせたもの。
    df_remain = df[df['id'].apply(lambda x:x not in all_got_ids)].reset_index(drop=True)
    return df_remain

In [11]:
# 分配の公平性を調べる。
def Judge_fairness(before_points, after_points):
    x_client = (np.array(after_points) - np.array(before_points)) / np.array(before_points)
    mean_array = np.full(len(x_client), x_client.mean())
    return sum(((x_client - mean_array)**2/len(x_client))**(1/2)/x_client.mean())

# 分配の効率を調べる。(増えた視聴率/枠数)
def Judge_efficiency(before_points, after_points, got_idses):
    ids_num = len(Flatten_dual(got_idses))
    profit = sum(np.array(after_points) - np.array(before_points))
    return profit/ids_num

# 分配の公平性を調べる。（枠の数） 単純に分散を出しているだけ。
def Judge_Number_fairness(before_ids, after_ids):
    before_id_num = [len(before_ids[i]) for i in range(len(before_ids))]
    after_id_num = [len(after_ids[i]) for i in range(len(after_ids))]
    return Judge_fairness(before_id_num, after_id_num)

In [12]:
# 2重のリストをフラットにする関数(重複は残る！)
def Flatten_dual(nested_list):
    return [e for inner_list in nested_list for e in inner_list]

### ○実際にアルゴリズムを動かす。

In [13]:
print('初めの枠数:', len(df_input))
start = time.time() # プログラム開始時間

GA_results = dict() # 公平性/効率性をkey, 手に入れたidをvalueとしたディクショナリ
for i in range(3): # 貪欲法のループ
    obtained_idses = [[] for i in range(client_num)] # 初期化する。
    GA_after_idses = Greedy_Algorithm(df_input, original_points, obtained_idses, client_num)
    GA_after_prices, GA_after_points = Check_pp(df_input, GA_after_idses, client_num)
    # amount_fairness = Judge_fairness(original_points, GA_after_points)
    # GA_results[amount_fairness] = GA_after_idses
    # efficiency = Judge_efficiency(original_points, GA_after_points, GA_after_idses)
    # GA_results[efficiency] = GA_after_idses
    number_fairness = Judge_Number_fairness(original_idses, GA_after_idses)
    GA_results[number_fairness] = GA_after_idses
    
GA_after_idses = GA_results[max(GA_results.keys())] # Greedy Algorithm で各クライアントが手に入れたidのリスト。
df_GAremain = Check_remainingdf(df_input, GA_after_idses) # 残っているデータフレーム  
GA_after_prices, GA_after_points = Check_pp(df_input, GA_after_idses, client_num) 

GA_end = time.time()
print('*'*17, '超貪欲法終了', '*'*17)
print('プログラム処理経過時間', round(GA_end-start,5), '[sec]')
print('残り枠数:', len(df_GAremain))

DP_results = dict() 
rate = 1.05
for i in range(3): # 動的計画法のループ
    DP_after_idses= Dynamic_Programming(df_GAremain, original_prices, GA_after_prices, rate, client_num)
    DP_after_prices, DP_after_points = Check_pp(df_GAremain, DP_after_idses, client_num)
    # amount_fairness = Judge_fairness(GA_after_points, DP_after_points)
    # DP_results[fairness] = DP_after_idses
    efficiency = Judge_efficiency(GA_after_points, DP_after_points, DP_after_idses)
    DP_results[efficiency] = DP_after_idses
    # number_fairness = Judge_Number_fairness(original_idses, DP_afteridses)
    # DP_results[number_fairness] = DP_after_idses
    
DP_after_idses = DP_results[max(DP_results.keys())]
df_DPremain = Check_remainingdf(df_GAremain, DP_after_idses) # 残っているデータフレーム  
DP_after_prices, DP_after_points = Check_pp(df_GAremain, DP_after_idses, client_num)

DP_end = time.time()
print('*'*16, '動的計画法終了', '*'*16)
print('プログラム処理経過時間', round(DP_end-start,5), '[sec]')
print('残り枠数:', len(df_DPremain))

total_prices = list(np.array(GA_after_prices) + np.array(DP_after_prices))
total_points = list(np.array(GA_after_points) + np.array(DP_after_points))
total_idses  = list(np.array(GA_after_idses)  + np.array(DP_after_idses))

profit = list(np.array(total_points) - np.array(original_points))
price_difference = list(np.array(total_prices) - np.array(original_prices))

np.set_printoptions(precision=3)
print('*' * 48)
print('利益　:', profit)
print('価格差:', price_difference)

初めの枠数: 5000
***************** 超貪欲法終了 *****************
プログラム処理経過時間 36.19792 [sec]
残り枠数: 320
**************** 動的計画法終了 ****************
プログラム処理経過時間 44.20408 [sec]
残り枠数: 0
************************************************
利益　: [688.2999999999975, 403.20000000000255, 314.8000000000029, 354.5999999999967, 325.5999999999967, 298.79999999999654, 257.50000000000273, 348.5000000000009, 429.8000000000029, 665.0000000000027]
価格差: [-6665, -5661, -3058, 6779, 4228, 2849, -5723, 746, -149, -386]


<b>ここからやらなければならないこと。
    1. 最適解にどれだけ早く近づいているかの測定。
    2. 実際のデータで動かしてみる。
    3. 価格を超えてしまった時の対処法。（処理時間と要相談）
    →ランダム性を加えるのが良いのでは？
    （遺伝的アルゴリズムとか、ランダム性が入ってる。）
    4. プログラムの見直し
</b>

## プロトタイプのモデルは完成！